In [199]:
import numpy as np
import keras
import os
import tensorflow as tf
import pandas as pd
import cv2

from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Sequential
from keras import backend as K

from PIL import Image

In [165]:
def preprocess(img):
    
    width, height = img.shape[0], img.shape[1]
    
    img = image.array_to_img(img, scale=False)
    
    desired_width, desired_height = 224, 224
    
    if width < desired_width:
        desired_width = width
    if height < desired_height:
        desired_height = height
    
    start_x = np.maximum(0, int((width-desired_width)//2))
    
    img = img.crop((start_x, np.maximum(0, height-desired_height), start_x + desired_width, height))
    
    img = img.resize((224, 224))
    
    img = image.img_to_array(img)
    
    return img / 255  
  

In [166]:
def get_training_images():
    train_folder = "data/news_imgs/train"
    
    files = os.listdir(train_folder)
    
    files = list(map(lambda x: os.path.join(train_folder, x), files)) # load all the training images
    
    # get started with just 20 images for training
    train_images = files[0:20]

    # preprocess images and load them
    images = []
    for item in train_images:
        img = image.load_img(item)
        img = image.img_to_array(img)
        img = preprocess(img)
        images.append(img)
    
    return images

In [200]:
def convert_labels_to_numpy_array(items): 
    values = []
    
    for index, row in items.iterrows():

        cats = {
                'protest': True,
                'violence': True,
                'sign': True,
                'photo': True,
                'fire': True,
                'police': True,
                'children': True,
                'group_20': True,
                'group_100': True,
                'flag': True,
                'night': True,
                'shouting': True
               }

        if row['protest']:
            
            array = []
            
            for cat in cats:
                f = float(row[cat])
                array.append(f)
            
            narray = np.array(array)
        
            values.append(narray)

    return values

In [168]:
def get_training_labels():
    
    labels = pd.read_csv("data/news_imgs/training.csv") # load csv of text data provided
    
    ll = convert_labels_to_numpy_array(labels)
 
    train_labels = ll[0:20]    # train only in 20 images first
    
    return train_labels

In [169]:
def reshape_images(images, size):
    return np.asarray(images).reshape(size,224,224,3)

In [209]:
def get_model(base_model):
    
    bottleneck_input = base_model.get_layer(index=0).input
    bottleneck_output = base_model.get_layer(index=-2).output
    
    bottleneck_model = Model(inputs=bottleneck_input, outputs=bottleneck_output)
    
    for layer in bottleneck_model.layers:
        
        layer.trainable = False
    
    new_model = Sequential()
    
    new_model.add(bottleneck_model)
    
    new_model.add(Dense(11, input_dim=2048, activation='softmax'))
    
    return new_model

In [208]:
base_model = InceptionV3()

new_model = get_model(base_model)

new_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [201]:
labels = get_training_labels()

images = reshape_images(get_training_images(), 20)

In [213]:
print(labels)

model.fit(images, labels[0], epochs=5, shuffle=True)

[array([1.        , 0.34870572, 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ]), array([1.        , 0.15315054, 1.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        ]), array([1.        , 0.52754146, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ]), array([1.        , 0.18533357, 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ]), array([1.        , 0.07231162, 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ]), array([1.        , 0.27033797, 1.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        ]

ValueError: Error when checking target: expected dense_54 to have shape (10,) but got array with shape (1,)

In [ ]:
# base_model = InceptionV3(weights='imagenet', include_top=False)

In [ ]:
# x = base_model.output

# x = GlobalAveragePooling2D()(x)

# x = Dense(2048, activation='relu')(x)

# # and a logistic layer -- let's say we have 10 classes
# predictions = Dense(10, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

# for layer in base_model.layers:
#     layer.trainable = False
    
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# model.fit(fimages, train_labels,
#                     steps_per_epoch=10000, epochs=10)

